# WCST
## Model Free Analysis

Statistical inquiry into the aggregate behaviour of the Wisconsin Sorting Task.


---------
```
Zach Wolpe
zachcolinwolpe@gmail.com
03 June 2021
```
---------



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os
import re
import sys
sys.path.append('../process data/')
import plotly.graph_objects as go
import plotly.express as px
from process_data_raw_class import batch_processing

In [2]:
with open('../data objects/batch_processing_object.pkl', 'rb') as file2:
    bp = pickle.load(file2)

In [3]:
bp.describe_data()



        ------------------------------------------------------------------
            self.path            : raw data loc
            self.metadata        : mturk metadata
            self.mapping         : reference table
            self.data_times      : reference times table
            self.participants    : list of participant identifiers
            self.parti_code      : list of participant codes
            self.n               : total number of samples
            self.wcst_paths      : paths to wcst  raw data
            self.nback_paths     : paths to nback raw data
            self.corsi_paths     : paths to corsi raw data
            self.fitts_paths     : paths to fitts raw data
            self.navon_paths     : paths to navon raw data
            self.wcst_data       : wcst  dataframe
            self.nback_data      : nback dataframe
            self.corsi_data      : corsi dataframe
            self.fitts_data      : fitts dataframe
            self.navon_data    

In [9]:
pd.options.display.max_rows = 4000
bp.wcst_data.head()

,participant,participant_code,card_no,correct_card,correct_persevering,seq_no,rule,card_shape,card_number,card_colour,reaction_time_ms,status,card_selected,error,perseverance_error,not_perseverance_error
0,816404.0,s.6463d918-7e29-49b0-942d-f1a9faab6ad6.txt,52,1,0,1,shape,triangle,1,yellow,3775,1,1,0,0,0
1,816404.0,s.6463d918-7e29-49b0-942d-f1a9faab6ad6.txt,59,3,0,2,shape,triangle,3,red,3771,1,3,0,0,0
2,816404.0,s.6463d918-7e29-49b0-942d-f1a9faab6ad6.txt,23,2,0,3,shape,cross,2,red,1359,1,2,0,0,0
3,816404.0,s.6463d918-7e29-49b0-942d-f1a9faab6ad6.txt,52,1,0,4,shape,triangle,1,yellow,1209,1,1,0,0,0
4,816404.0,s.6463d918-7e29-49b0-942d-f1a9faab6ad6.txt,52,1,0,5,shape,triangle,1,yellow,953,1,1,0,0,0


In [ ]:
# ---- probability of correct ----x
x_bar = bp.wcst_data.groupby(['seq_no'])['reaction_time_ms'].mean()
st    = bp.wcst_data.groupby(['seq_no'])['reaction_time_ms'].std()


In [31]:
# ---- reaction time ----x
x_bar = bp.wcst_data.groupby(['seq_no'])['reaction_time_ms'].mean()
st    = bp.wcst_data.groupby(['seq_no'])['reaction_time_ms'].std()


import plotly.graph_objects as go
import numpy as np



x     = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
x_rev = x[::-1]

# Line 1
y1 = x_bar
y1_upper = x_bar - st
y1_lower = x_bar + st
y1_lower = y1_lower[::-1]





fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x+x_rev,
    y=y1_upper+y1_lower-y1,
    fill='toself',
    fillcolor='rgba(0,176,246,0.2)',
    line_color='rgba(255,255,255,0)',
    showlegend=False,
    name='Fair',
))

fig.add_trace(go.Scatter(
    x=x, y=y1,
    line_color='#darkred',
    name='response time (ms)',
))

fig.update_traces(mode='lines')
fig.update_layout(title='wha')
fig.show()
def pie_chart(dummy_var, labels, colors, title):
    sub    = df[[dummy_var]].value_counts()
    values = sub.tolist()
    fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.4)])
    fig.update_traces(textfont_size=15, marker=dict(colors=colors, line=dict(color='white', width=0)))
    fig.update(layout_title_text=title)
    fig.show()


In [30]:

st

seq_no
1     1865.110482
2     1882.918611
3     1749.736210
4     1884.010153
5     1810.027556
6     1786.411812
7     1750.554837
8     1892.071608
9     1630.385682
10    1865.273328
Name: reaction_time_ms, dtype: float64